In [1]:
# Import required libraries
import sagemaker
import pandas as pd
import numpy as np
import boto3, io, json

from scipy.sparse import lil_matrix
from sagemaker import get_execution_role
from sagemaker.predictor import json_deserializer
from sagemaker.amazon.common import write_spmatrix_to_sparse_tensor
from sagemaker.amazon.amazon_estimator import get_image_uri

In [2]:
# Set sagemaker environment variables
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

INFO:sagemaker:Created S3 bucket: sagemaker-eu-west-1-800020257645


In [3]:
# Set paths to datasets
df_location = 's3://' + bucket + "/Dataset/UserArtistIDDataset.csv"
train_location = 's3://' + bucket + "/Dataset/TrainDataset.csv"
test_location = 's3://' + bucket + "/Dataset/TestDataset.csv"

# Read the datasets into dataframes
df = pd.read_csv(df_location, sep=',', names=['user_id', 'artist_id', 'track_count'], encoding="utf8")
train = pd.read_csv(train_location, sep=',', names=['user_id', 'artist_id', 'track_count'], encoding="utf8")
test = pd.read_csv(test_location, sep=',', names=['user_id', 'artist_id', 'track_count'], encoding="utf8")

In [6]:
test.head()

,user_id,artist_id,track_count
0,4619,13865,1
1,6353,27558,1
2,11840,16025,1
3,5797,11888,5
4,6807,13585,1


In [9]:
# Get the total number of users, artist and features
numUsers = df.user_id.max()
numArtists = df.artist_id.max()
numFeatures = numUsers + numArtists
print("#Users: ", numUsers, "\n#Artists: ", numArtists, "\n#Features: ", numFeatures)

#Users:  14601 
#Artists:  33588 
#Features:  48189


In [10]:
# Set up the sparse one hot encoded matrix. This creates a sparse matrix of 1's and 0's. 1 for each artist the user
# has 4 or more tracks for, indicating a strong preference, 0 for all other.
def one_hot_encode(df):
    feature_matrix = lil_matrix((df.shape[0], numFeatures)).astype('float32')
    label_matrix = []

    rowNum = 0
    for index, row in df.iterrows():
        feature_matrix[rowNum, int(row['user_id']) - 1] = 1
        feature_matrix[rowNum, int(numUsers) + int(row['artist_id']) - 1] = 1
        if int(row['track_count']) >= 4:
            label_matrix.append(1)
        else:
            label_matrix.append(0)
        rowNum += 1
    
    label_matrix = np.array(label_matrix).astype('float32')
    return feature_matrix, label_matrix


# One hot encode the datasets
full_feature, full_label = one_hot_encode(df)
train_feature, train_label = one_hot_encode(train)
test_feature, test_label = one_hot_encode(test)

In [11]:
# Covert the dataset into protobuf format. The required format of the factorization machine algo 
def convert_df_protobuf_write(feature, label, name):
    buf = io.BytesIO()
    write_spmatrix_to_sparse_tensor(buf, feature, label)
    buf.seek(0)
    boto3.resource('s3').Bucket(bucket).Object(name).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket, name)


# Convert all of the datasets
allData_file = convert_df_protobuf_write(full_feature, full_label, 'full.protobuf')
trainData_file = convert_df_protobuf_write(train_feature, train_label, 'train.protobuf')
testData_file = convert_df_protobuf_write(test_feature, test_label, 'test.protobuf')

In [15]:
allData_file = 's3://{}/{}'.format(bucket, 'full.protobuf')
trainData_file = 's3://{}/{}'.format(bucket, 'train.protobuf')
testData_file = 's3://{}/{}'.format(bucket, 'test.protobuf')

In [12]:
# Load the factorization machine algo from the amazon container
container = get_image_uri(boto3.Session().region_name, 'factorization-machines')

# Set the location the model will be stored at
output_file = 's3://{}/Output'.format(bucket)

In [13]:
# Initialize the model and set parameters
fm = sagemaker.estimator.Estimator(container,
                                   get_execution_role(),
                                   train_instance_count = 1,
                                   train_instance_type = 'ml.c4.xlarge',
                                   output_path = output_file,
                                   sagemaker_session = sess)

# Set the hyperparameters for the model
fm.set_hyperparameters(feature_dim = numFeatures,
                       predictor_type = 'binary_classifier',
                       mini_batch_size = 1000,
                       num_factors = 64,
                       epochs = 100)

In [16]:
# Fit the model
fm.fit({'train': trainData_file, 'test': testData_file})

INFO:sagemaker:Creating training-job with name: factorization-machines-2019-03-25-18-55-44-623


2019-03-25 18:55:44 Starting - Starting the training job...
2019-03-25 18:55:45 Starting - Launching requested ML instances......
2019-03-25 18:56:47 Starting - Preparing the instances for training...
2019-03-25 18:57:36 Downloading - Downloading input data
2019-03-25 18:57:36 Training - Downloading the training image....
Docker entrypoint called with argument(s): train
[03/25/2019 18:58:13 INFO 140146317055808] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_sigma': u'0.001', u'_log_level': u'info', u'bias_init_method': u'normal', u'linear_init_method': u'normal', u'linear_lr': u'0.001', u'factors_init_method': u'normal', u'_tuning_objective_metric': u'', u'bias_wd': u'0.01', u'use_linear': u'true', u'bias_lr': u'0.1', u'mini_batch_size': u'1000', u'_use_full_symbolic'

[03/25/2019 18:58:47 INFO 140146317055808] Iter[1] Batch [1500]#011Speed: 118676.00 samples/sec
[03/25/2019 18:58:47 INFO 140146317055808] #quality_metric: host=algo-1, epoch=1, batch=1500 train binary_classification_accuracy <score>=0.805967355097
[03/25/2019 18:58:47 INFO 140146317055808] #quality_metric: host=algo-1, epoch=1, batch=1500 train binary_classification_cross_entropy <loss>=0.467183761221
[03/25/2019 18:58:47 INFO 140146317055808] #quality_metric: host=algo-1, epoch=1, batch=1500 train binary_f_1.000 <score>=0.00826775541337
[03/25/2019 18:58:51 INFO 140146317055808] Iter[1] Batch [2000]#011Speed: 121389.83 samples/sec
[03/25/2019 18:58:51 INFO 140146317055808] #quality_metric: host=algo-1, epoch=1, batch=2000 train binary_classification_accuracy <score>=0.805784607696
[03/25/2019 18:58:51 INFO 140146317055808] #quality_metric: host=algo-1, epoch=1, batch=2000 train binary_classification_cross_entropy <loss>=0.466605473036
[03/25/2019 18:58:51 INFO 140146317055808] #quali

[03/25/2019 18:59:25 INFO 140146317055808] Iter[3] Batch [1000]#011Speed: 119520.58 samples/sec
[03/25/2019 18:59:25 INFO 140146317055808] #quality_metric: host=algo-1, epoch=3, batch=1000 train binary_classification_accuracy <score>=0.80871028971
[03/25/2019 18:59:25 INFO 140146317055808] #quality_metric: host=algo-1, epoch=3, batch=1000 train binary_classification_cross_entropy <loss>=0.451666382507
[03/25/2019 18:59:25 INFO 140146317055808] #quality_metric: host=algo-1, epoch=3, batch=1000 train binary_f_1.000 <score>=0.0451397026923
[03/25/2019 18:59:29 INFO 140146317055808] Iter[3] Batch [1500]#011Speed: 119612.15 samples/sec
[03/25/2019 18:59:29 INFO 140146317055808] #quality_metric: host=algo-1, epoch=3, batch=1500 train binary_classification_accuracy <score>=0.809282478348
[03/25/2019 18:59:29 INFO 140146317055808] #quality_metric: host=algo-1, epoch=3, batch=1500 train binary_classification_cross_entropy <loss>=0.450566309065
[03/25/2019 18:59:29 INFO 140146317055808] #quality

[03/25/2019 19:00:07 INFO 140146317055808] Iter[5] Batch [1000]#011Speed: 119030.10 samples/sec
[03/25/2019 19:00:07 INFO 140146317055808] #quality_metric: host=algo-1, epoch=5, batch=1000 train binary_classification_accuracy <score>=0.812607392607
[03/25/2019 19:00:07 INFO 140146317055808] #quality_metric: host=algo-1, epoch=5, batch=1000 train binary_classification_cross_entropy <loss>=0.441276615749
[03/25/2019 19:00:07 INFO 140146317055808] #quality_metric: host=algo-1, epoch=5, batch=1000 train binary_f_1.000 <score>=0.101163437027
[03/25/2019 19:00:11 INFO 140146317055808] Iter[5] Batch [1500]#011Speed: 120559.75 samples/sec
[03/25/2019 19:00:11 INFO 140146317055808] #quality_metric: host=algo-1, epoch=5, batch=1500 train binary_classification_accuracy <score>=0.813055296469
[03/25/2019 19:00:11 INFO 140146317055808] #quality_metric: host=algo-1, epoch=5, batch=1500 train binary_classification_cross_entropy <loss>=0.440404030329
[03/25/2019 19:00:11 INFO 140146317055808] #quality

...

[03/25/2019 19:30:49 INFO 140146317055808] Iter[93] Batch [1000]#011Speed: 120459.14 samples/sec
[03/25/2019 19:30:49 INFO 140146317055808] #quality_metric: host=algo-1, epoch=93, batch=1000 train binary_classification_accuracy <score>=0.844632367632
[03/25/2019 19:30:49 INFO 140146317055808] #quality_metric: host=algo-1, epoch=93, batch=1000 train binary_classification_cross_entropy <loss>=0.365280385904
[03/25/2019 19:30:49 INFO 140146317055808] #quality_metric: host=algo-1, epoch=93, batch=1000 train binary_f_1.000 <score>=0.429367627364
[03/25/2019 19:30:53 INFO 140146317055808] Iter[93] Batch [1500]#011Speed: 119486.53 samples/sec
[03/25/2019 19:30:53 INFO 140146317055808] #quality_metric: host=algo-1, epoch=93, batch=1500 train binary_classification_accuracy <score>=0.84461892072
[03/25/2019 19:30:53 INFO 140146317055808] #quality_metric: host=algo-1, epoch=93, batch=1500 train binary_classification_cross_entropy <loss>=0.364947103034
[03/25/2019 19:30:53 INFO 140146317055808] #q

[03/25/2019 19:31:26 INFO 140146317055808] Iter[95] Batch [500]#011Speed: 119975.77 samples/sec
[03/25/2019 19:31:26 INFO 140146317055808] #quality_metric: host=algo-1, epoch=95, batch=500 train binary_classification_accuracy <score>=0.845387225549
[03/25/2019 19:31:26 INFO 140146317055808] #quality_metric: host=algo-1, epoch=95, batch=500 train binary_classification_cross_entropy <loss>=0.363834016598
[03/25/2019 19:31:26 INFO 140146317055808] #quality_metric: host=algo-1, epoch=95, batch=500 train binary_f_1.000 <score>=0.431741653401
[03/25/2019 19:31:31 INFO 140146317055808] Iter[95] Batch [1000]#011Speed: 118626.89 samples/sec
[03/25/2019 19:31:31 INFO 140146317055808] #quality_metric: host=algo-1, epoch=95, batch=1000 train binary_classification_accuracy <score>=0.845184815185
[03/25/2019 19:31:31 INFO 140146317055808] #quality_metric: host=algo-1, epoch=95, batch=1000 train binary_classification_cross_entropy <loss>=0.364169059212
[03/25/2019 19:31:31 INFO 140146317055808] #qual

[03/25/2019 19:32:17 INFO 140146317055808] Iter[97] Batch [1500]#011Speed: 120534.30 samples/sec
[03/25/2019 19:32:17 INFO 140146317055808] #quality_metric: host=algo-1, epoch=97, batch=1500 train binary_classification_accuracy <score>=0.845704863424
[03/25/2019 19:32:17 INFO 140146317055808] #quality_metric: host=algo-1, epoch=97, batch=1500 train binary_classification_cross_entropy <loss>=0.362729610009
[03/25/2019 19:32:17 INFO 140146317055808] #quality_metric: host=algo-1, epoch=97, batch=1500 train binary_f_1.000 <score>=0.434236465838
[03/25/2019 19:32:21 INFO 140146317055808] Iter[97] Batch [2000]#011Speed: 119788.25 samples/sec
[03/25/2019 19:32:21 INFO 140146317055808] #quality_metric: host=algo-1, epoch=97, batch=2000 train binary_classification_accuracy <score>=0.845428785607
[03/25/2019 19:32:21 INFO 140146317055808] #quality_metric: host=algo-1, epoch=97, batch=2000 train binary_classification_cross_entropy <loss>=0.362956898519
[03/25/2019 19:32:21 INFO 140146317055808] #

[03/25/2019 19:32:50 INFO 140146317055808] Iter[99] Batch [500]#011Speed: 118703.41 samples/sec
[03/25/2019 19:32:50 INFO 140146317055808] #quality_metric: host=algo-1, epoch=99, batch=500 train binary_classification_accuracy <score>=0.846283433134
[03/25/2019 19:32:50 INFO 140146317055808] #quality_metric: host=algo-1, epoch=99, batch=500 train binary_classification_cross_entropy <loss>=0.361621766355
[03/25/2019 19:32:50 INFO 140146317055808] #quality_metric: host=algo-1, epoch=99, batch=500 train binary_f_1.000 <score>=0.437482652331
[03/25/2019 19:32:54 INFO 140146317055808] Iter[99] Batch [1000]#011Speed: 118006.58 samples/sec
[03/25/2019 19:32:54 INFO 140146317055808] #quality_metric: host=algo-1, epoch=99, batch=1000 train binary_classification_accuracy <score>=0.846140859141
[03/25/2019 19:32:54 INFO 140146317055808] #quality_metric: host=algo-1, epoch=99, batch=1000 train binary_classification_cross_entropy <loss>=0.361941744382
[03/25/2019 19:32:54 INFO 140146317055808] #qual

Billable seconds: 2208


In [17]:
# Deploy the model
fm_predictor = fm.deploy(instance_type = 'ml.c4.xlarge', initial_instance_count = 1)

INFO:sagemaker:Creating model with name: factorization-machines-2019-03-25-19-36-34-420
INFO:sagemaker:Creating endpoint with name factorization-machines-2019-03-25-18-55-44-623


---------------------------------------------------------------------------!

In [46]:
# Serialize the data returned by the model
def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    print(js)
    return json.dumps(js)


# Configure the endpoints serialization methods
fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

In [48]:
# Make some predictions
result = fm_predictor.predict(test_feature[1000].toarray())
print(result)
print(test_label[9000:9010])

{'instances': [{'features': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [45]:
print(test_feature[9010])

  (0, 10159)	1.0
  (0, 16234)	1.0


In [49]:
# Delete endpoint
sess.delete_endpoint(fm_predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: factorization-machines-2019-03-25-18-55-44-623
